# Political blogs as networks in the 2004 election

Introductory text, blah blah

Load esssential libraries:

In [9]:
from urllib.request import urlopen

import networkx as nx


## Load the data

Explain the data some...

Source: <http://networkdata.ics.uci.edu/data/polblogs/>

In [18]:
glm_text = urlopen('http://networkdata.ics.uci.edu/data/polblogs/polblogs.gml').read().decode('utf-8')
glm = glm_text.split('\n')

G = nx.parse_gml(glm)
G.size()

NetworkXError: edge #12757 (1047->1179) is duplicated.
Hint: If multigraph add "multigraph 1" to file header.

In [29]:
glm_txt = open('polblogs.glm.txt').read()

glm = glm_txt.split('\n')
glm[0:10]

['Creator "Lada Adamic on Tue Aug 15 2006"',
 'graph [',
 '  directed 1',
 '  node [',
 '    id 1',
 '    label "100monkeystyping.com"',
 '    value 0',
 '    source "Blogarama"',
 '  ]',
 '  node [']

In [44]:
G = nx.readwrite.read_gml('polblogs.gml', label='label')
G.size()

19090